# 手把手搭建 seq2seq（中）：基于注意力机制的实现

## 注意力机制

<img src="assets/attention-output.svg" width="50%" height="50%" align="left"/>

* *注意力机制*本质上是一种*汇聚*（attention pooling）操作，相对于全连接层或汇聚层增加了query查询过程。
* 注意力汇聚操作可以表示成值的加权和。选择不同的注意力评分函数会带来不同的注意力汇聚操作。
* *注意力评分函数*（attention scoring function）经过softmax运算得到*注意力权重*（attention weights）。

### 注意力机制的数学描述

假设有一个查询
$\mathbf{q} \in \mathbb{R}^q$和
$m$个“键－值”对
$(\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)$，
其中$\mathbf{k}_i \in \mathbb{R}^k$，$\mathbf{v}_i \in \mathbb{R}^v$。
**注意力汇聚函数$f$**就被表示成值的加权和：

$$f(\mathbf{q}, (\mathbf{k}_1, \mathbf{v}_1), \ldots, (\mathbf{k}_m, \mathbf{v}_m)) = \sum_{i=1}^m \alpha(\mathbf{q}, \mathbf{k}_i) \mathbf{v}_i \in \mathbb{R}^v,$$


其中查询$\mathbf{q}$和键$\mathbf{k}_i$的注意力权重（标量）
是通过**注意力评分函数$a$** 将两个向量映射成标量，
再经过softmax运算得到的：

$$\alpha(\mathbf{q}, \mathbf{k}_i) = \mathrm{softmax}(a(\mathbf{q}, \mathbf{k}_i)) = \frac{\exp(a(\mathbf{q}, \mathbf{k}_i))}{\sum_{j=1}^m \exp(a(\mathbf{q}, \mathbf{k}_j))} \in \mathbb{R}.$$


## 注意力模型实现


选择不同的注意力评分函数$a$会导致不同的注意力汇聚操作。下面介绍两种流行的注意力模型实现。


### Additive Attention

Additive Attention的评分函数
$$a(\mathbf q, \mathbf k) = \mathbf w_v^\top \text{tanh}(\mathbf W_q\mathbf q + \mathbf W_k \mathbf k) \in \mathbb{R}.$$

其中可学习的参数是$\mathbf W_q\in\mathbb R^{h\times q}$、
$\mathbf W_k\in\mathbb R^{h\times k}$和
$\mathbf w_v\in\mathbb R^{h}$。


* 可以看作将查询和键连结起来后输入到一个多层感知机（MLP）中，包含一个使用$\tanh$作为激活函的隐藏层。
* 适用于查询和键长度不同时的情形。

### Scaled dot-product Attention

Scaled dot-product Attention的评分函数
$$a(\mathbf q, \mathbf k) = \mathbf{q}^\top \mathbf{k}  /\sqrt{d}.$$

不需要可学习的参数。

在实践中，通常从小批量的角度来考虑提高效率，
例如基于$n$个查询和$m$个键－值对计算注意力，
其中查询和键的长度为$d$，值的长度为$v$。
查询$\mathbf Q\in\mathbb R^{n\times d}$、
键$\mathbf K\in\mathbb R^{m\times d}$和
值$\mathbf V\in\mathbb R^{m\times v}$的缩放点积注意力是：

$$ \mathrm{softmax}\left(\frac{\mathbf Q \mathbf K^\top }{\sqrt{d}}\right) \mathbf V \in \mathbb{R}^{n\times v}.$$





* 适用于查询和键是长度相同时的情形，使用Scaled dot-product Attention 计算效率更高。
* 假设查询和键的所有元素都是独立的随机变量，
  并且都满足零均值和单位方差，
  那么两个向量的点积的均值为$0$，方差为$d$。
  为确保无论向量长度如何，
  点积的方差在不考虑向量长度的情况下仍然是$1$，
  将点积除以$\sqrt{d}$，

### 代码实现


[代码实现详见](https://github.com/gnudennis/deepnlp/blob/master/core/translation/models/attention.py)


实现细节：

* 计算注意力权重时采用掩码注意力。
* 注意力权重与值加权和通常使用dropout正则化。 

## 基于注意力模型的实现

基于循环神经网络的实现的基础上，增加注意力机制。采用Bahdanau（cite:`Bahdanau.Cho.Bengio.2014`）的方案。

<img src="assets/seq2seq-attention-details.svg" width="40%" height="40%" align="left"/>

#### 实现细节

1. Encoder可以采用双向神经网络实现。
2. Bahdanau注意力将上一时间步的解码器隐状态视为查询，在所有时间步的编码器隐状态同时视为键和值。
3. 注意力机制的输出同时作为Decoder隐藏层和输出层的输入。


<img src="assets/seq2seq-attn.png" width="40%" height="40%" align="left"/>

## references

* [d2l](https://zh-v2.d2l.ai/chapter_attention-mechanisms/attention-scoring-functions.html)
* [pytorch-seq2seq](https://github.com/bentrevett/pytorch-seq2seq)